In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import warnings
import sys
from sklearn.metrics import pairwise_distances, mean_absolute_error, mean_squared_error
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

import threading

data_base_dir = '../../../../datasets/Movielens/'
data_dir2 = data_base_dir + 'serendipity-sac2018/'
data_dir = data_base_dir + 'ml-20m/'
output_dir = data_dir + 'output2/'

genome_scores = data_dir + 'genome-scores.csv'
# genome_scores = data_dir + 'tag_genome.csv'
genome_tags = data_dir + 'genome-tags.csv'
movies = data_dir + 'movies.csv'
ratings = data_dir + 'ratings.csv'

# data loading and preprocessing
target_df = pd.read_csv(genome_scores).pivot(index='movieId', columns='tagId',
                                             values='relevance')
genome_scores_df = pd.DataFrame(pairwise_distances(target_df, metric='cosine'),
                                index=target_df.index,
                                columns=target_df.index)
del target_df

movies_with_genome = genome_scores_df.index.values

movies_df = pd.read_csv(movies)
movies_df = movies_df[movies_df['genres'] != '(no genres listed)']
movies_df = movies_df[movies_df['movieId'].isin(movies_with_genome)]
del movies_with_genome

all_movie_ids = movies_df['movieId'].unique()
del movies_df

ratings_df = pd.read_csv(ratings)
ratings_df = ratings_df[ratings_df['movieId'].isin(all_movie_ids)]
ratings_df = ratings_df.loc[:, ['userId', 'movieId', 'rating']]

all_user_ids = ratings_df['userId'].unique()
count_df = ratings_df.groupby('userId').count()
# count_df.describe()

# # divide user groups into 4 based on the number of movies watched by them
threshold1 = count_df['movieId'] <= 34
threshold2 = count_df['movieId'] <= 67
threshold3 = count_df['movieId'] <= 154
threshold4 = count_df['movieId'] > 154

user_group_1 = count_df[threshold1].index.values
user_group_2 = count_df[threshold2].index.values
user_group_3 = count_df[threshold3].index.values
user_group_4 = count_df[threshold4].index.values

del count_df

user_group_2 = np.setdiff1d(user_group_2, user_group_1)
user_group_3 = np.setdiff1d(user_group_3, user_group_2)
user_group_4 = np.setdiff1d(user_group_4, user_group_3)

print(user_group_1.size)
print(user_group_2.size)
print(user_group_3.size)
print(user_group_4.size)

34763
34545
69462
34486


In [4]:
def print_progress(user_group, user_id):
    print('Progress %.2f%%' % ((np.where(user_group == user_id)[0][0]/(user_group.size - 1)) * 100))

In [8]:
print_progress(user_group_4, 347)

Progress 0.23%
